In [30]:
%pip install haversine

DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/geopandas-0.9.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
!ls

CSV Attribute Table Exports
Daily_Census_Tract-Level_PM2.5_Concentrations__2011-2014_20240413.csv
LICENSE
ca_daily_fire_2000_2021.csv
factories.ipynb
fireforce.ipynb
test.txt


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from haversine import haversine, Unit

/var/folders/19/r56dh6d15dvbmqph7x4f4r3m0000gn/T/ipykernel_62320/3125623339.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
fire = pd.read_csv("./ca_daily_fire_2000_2021.csv")
fire.head()

,latitude,longitude,acq_date,satellite,instrument,confidence,year,month
0,32.436893,-114.597054,2014-06-15,N,VIIRS,100,2014,6
1,32.456921,-114.598595,2014-06-15,N,VIIRS,100,2014,6
2,32.457344,-114.596558,2014-06-15,N,VIIRS,100,2014,6
3,32.458092,-114.592819,2014-06-15,N,VIIRS,100,2014,6
4,32.461948,-114.593880,2014-06-15,N,VIIRS,100,2014,6


In [4]:
pm = pd.read_csv("./Daily_Census_Tract-Level_PM2.5_Concentrations__2011-2014_20240413.csv")
pm.head()


,year,date,statefips,countyfips,ctfips,latitude,longitude,ds_pm_pred,ds_pm_stdd
0,2011,30DEC2011,48,48399,48399950100,31.96861,-99.99100,7.590561,5.439812
1,2011,30DEC2011,48,48399,48399950200,31.95574,-99.96764,7.660033,5.666294
2,2011,30DEC2011,48,48399,48399950500,31.65529,-100.05925,7.355021,5.490203
3,2011,30DEC2011,48,48399,48399950600,31.76387,-99.89893,7.436393,5.247210
4,2011,30DEC2011,48,48401,48401950100,32.31673,-94.60574,11.107991,6.297006


In [1]:
fire.columns

NameError: name 'fire' is not defined

In [9]:
pm.columns

Index(['year', 'date', 'statefips', 'countyfips', 'ctfips', 'latitude',
       'longitude', 'ds_pm_pred', 'ds_pm_stdd'],
      dtype='object')

In [5]:
fire["acq_date"] = pd.to_datetime(fire['acq_date'])
pm['date'] = pd.to_datetime(pm['date'])

/var/folders/19/r56dh6d15dvbmqph7x4f4r3m0000gn/T/ipykernel_62320/3723032135.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pm['date'] = pd.to_datetime(pm['date'])


In [9]:
precision = 3  
fire['rounded_latitude'] = fire['latitude'].round(precision)
fire['rounded_longitude'] = fire['longitude'].round(precision)
pm['rounded_latitude'] = pm['latitude'].round(precision)
pm['rounded_longitude'] = pm['longitude'].round(precision)

In [10]:
merged_df = pd.merge(pm, fire, how='left', left_on=['rounded_latitude', 'rounded_longitude', 'date'], right_on=['rounded_latitude', 'rounded_longitude', 'acq_date'])
merged_df.head()

In [1]:
pwd

'/Users/renesmeey/Data102/emberalert-1'

In [ ]:
def calculate_distance(lat1, lon1, lat2, lon2):
    return haversine((lat1, lon1), (lat2, lon2))

def categorize_distance(distance):
    if distance < 10:
        return '0-10 km'
    elif distance < 50:
        return '10-50 km'
    elif distance < 100:
        return '50-100 km'
    else:
        return '100+ km'

# Calculate distances for each row in the merged dataset
merged_df['distance'] = merged_df.apply(
    lambda row: calculate_distance(row['latitude_x'], row['longitude_x'], row['latitude_y'], row['longitude_y']),
    axis=1
)

# Categorize distances
merged_df['distance_category'] = merged_df['distance'].apply(categorize_distance)

# Remove unnecessary columns for visualization
cleaned_df = merged_df[['date', 'ds_pm_pred', 'distance_category']]

# Group by date and distance category to calculate average PM 2.5 levels
final_df = cleaned_df.groupby(['date', 'distance_category']).mean().reset_index()

# Print final prepared DataFrame
print(final_df.head())


In [ ]:
# Assuming df_fire and df_pm are your pre-loaded datasets
# You would need to preprocess these datasets as described above.

# Example of plotting
plt.figure(figsize=(10, 6))
for category in distance_categories:
    subset = df_pm[df_pm['distance_category'] == category]
    plt.plot(subset['date'], subset['ds_pm_pred'], label=f'Distance: {category} km')

plt.xlabel('Date')
plt.ylabel('PM 2.5 Levels')
plt.title('PM 2.5 Levels Over Time by Proximity to Wildfire Events')
plt.legend()
plt.show()